# App Data

> Class that loads and stores the data and sidebar for the dash app

In [ ]:
#| default_exp app_data

In [ ]:
# TODO think about what differentiates dash_components and app_data

In [ ]:
#| export
from nbdev.showdoc import *
# TODO work out how to get around below hack
try:
    from SolomonIslandsDataMap.load_data import *
    from SolomonIslandsDataMap.dash_components import *
except: 
    from load_data import *
    from dash_components import *
from fastcore.test import *
from dash import dcc, html
import dash_bootstrap_components as dbc 
import dash_mantine_components as dmc

## Load Data

In [ ]:
#| export
# TODO data should be stored in store (or ) then loaded into pages.
sol_geo = SolomonGeo.load_pickle(aws = True)
geo_df = sol_geo.geo_df
fig = define_map(sol_geo)
# TODO save geo_df in store
#dcc.Store(id = 'selection',data = {})
geos = sol_geo.geo_levels
cen_vars = sol_geo.census_vars
NUM_GEOS = len(geos)

## Instatialise Data and Dash Components

The plan for the data is to store it in a dcc.Store object. This is import as if the app scales, variables will only be global if they are stored in a store or another dash component. As an aside, dash uses this strategy so that each app is stateless and multiple different workers can be assigned to run processes. 

This stored data and additional components will then be refereced via imports used by each page so that each pages shares the common data and components and they will not need to be reloaded. Because the data is imported as a module by each page and app that uses them, it will be global and it will run before the rest of the code. 

`SolomonGeo.gen_stored()` can be used to reconstuct the `SolomonGeo` object from the dict format.

The  following components and data objects are created:

- stored_data: The dict version of the dataset,
- dropdown_location: Dropdown for location selection,
- dropdown_geo: Dropdown for the type of geographical aggregation selected,
- control_type: The type of data, Proportion or Total, 
- dd_var: Dropdown for the variable to be displayed, 
- dd_measure: Dropdown for the emasure to be displayed.

In [ ]:
#| export
stored_data = dcc.Store(id="geo_df", data={"data-frame": geo_df.to_dict("series")})

In [ ]:
stored_data

Store(id='geo_df', data={'data-frame': {('core', 'geometry'): pk
Choiseul_Total         MULTIPOLYGON (((157.56463 -7.49272, 157.56005 ...
Choiseul_Proportion    MULTIPOLYGON (((157.56463 -7.49272, 157.56005 ...
Honiara_Total          MULTIPOLYGON (((159.91806 -9.42379, 159.92354 ...
Honiara_Proportion     MULTIPOLYGON (((159.91806 -9.42379, 159.92354 ...
Western_Total          MULTIPOLYGON (((158.25148 -8.78861, 158.24885 ...
                                             ...                        
Vanikoro_Proportion    MULTIPOLYGON (((166.98326 -11.67945, 166.98258...
Tikopia_Total          MULTIPOLYGON (((168.84036 -12.28504, 168.82691...
Tikopia_Proportion     MULTIPOLYGON (((168.84036 -12.28504, 168.82691...
Neo_Total              MULTIPOLYGON (((165.81001 -10.65326, 165.79693...
Neo_Proportion         MULTIPOLYGON (((165.81001 -10.65326, 165.79693...
Name: (core, geometry), Length: 486, dtype: geometry, ('core', 'id'): pk
Choiseul_Total           1
Choiseul_Proportion      1
Honia

Setup and create the dropdowns

In [ ]:
#| export
dropdown_location = html.Div(children = gen_dd(sol_geo.locations[sol_geo.geo_levels[0]], 
                                                'locDropdown', clear = True, place_holder='Select Dropdown Location',
                                                multi = True))

dropdown_geo = dmc.SegmentedControl(
                            id="segmented_geo",
                            value=geos[0],
                            data=geos,
                             orientation="vertical",
                            color = 'gray',
                            fullWidth = True,) # TODO consider redoing as theme is not consistent with this library
control_type = dmc.SegmentedControl(
                        id="segmented_type",
                        value=sol_geo.data_type[0],
                        data=sol_geo.data_type,
                        orientation="vertical",
                        color = 'gray',
                        fullWidth = True,)

dd_var = html.Div(children = gen_dd(list(sol_geo.census_vars.keys()), 'varDropdown', 
                                    val = 'Key Statistics', height = 75))
dd_measure = html.Div(children = gen_dd(sol_geo.census_vars['Key Statistics'], 'measureDropdown',
                                      val = sol_geo.census_vars['Key Statistics'][0]))

## Create a sidebar
The sidebar will 

In [ ]:
#| export
# Note, for now I am not using a sidebar style as I do not want to fix the width
# TODO fix the width of the sidebar, particular on different screens
SIDEBAR_STYLE = {
    "position": "fixed",
    "top": 0,
    "left": 0,
    "bottom": 0,
    "width": "24rem",
    "padding": "2rem 1rem",
    #"background-color": "#f8f9fa",
}


sidebar = html.Div(
    [
        html.H2("Filters"),
        html.Hr(),
        dbc.Nav(
            [
                html.P("Geography"), # TODO add a tooltip button here with link to geo explanation
                dropdown_geo,
                html.Br(),
                html.P("Location"), # TODO add a little info button here with link to geo explanation
                dropdown_location,
                html.Br(),
                html.P("Data"), # TODO add a little info button here with link to geo explanation
                dd_var,
                dd_measure,
                html.Br(),
                html.P("Data Type"), 
                control_type,
                #html.Br(),
                #dcc.Dropdown(id = 'three')

            ],
            vertical=True,
            pills=True,
        ),
    ],
    #style=SIDEBAR_STYLE,
)


## Intialise components specific to map_page
It will create:

- mytitle: title for the apge
- map_graph: map object for the page
- selectedBarGraph: bar graph object for the page

In [ ]:
#| export
# TODO - not sure whether this should be imported from app_data or built here.
# if building it here causes it to reload each time, I should probably move it later
# TODO downside of having it here is that it is a little more seperated.
mytitle = dcc.Markdown(children="## " + list(cen_vars.keys())[0] + " by " + geos[0]) # TODO This needs a default title
map_graph = dcc.Graph(figure=define_map(sol_geo), selectedData=None,)

selectedBarGraph = dcc.Graph(figure = gen_bar_plot(sol_geo, sol_geo.geo_levels[0], 
                                               "Key Statistics", 'Total Households'),
                            id = 'bar_graph')

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()